###### Para executar esse script você precisa ir ao site da bovespa e pegar algumas bases de dados históricas do ano que você desejar, no momento da criação desse script eram encontradas neste link (https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/) ou pode se pesquisar bovespa bases de dados históricas no google. Após isso carregue os arquivos para o google collab clicando no icone a esquerda que se parece com uma pasta e se chama 'files' e então clique no icone que contêm uma pasta com uma flecha para cima e selecione os arquivos no diretório de sua máquina, espere os arquivo serem totalmente carregados e rode o script.

In [ ]:
import pandas as pd
!pip install fsspec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def read_files(path, name_file, year_date, type_file):
  
  _file = f'{path}{name_file}{year_date}.{type_file}'

  colspecs = [(2,10),
              (10,12),
              (12,24),
              (27,39),
              (56,69),
              (69,82),
              (82,95),
              (108,121),
              (152,170),
              (170,188)
  ]

  names = ['data_pregao','codbdi','sigla_acao','nome_acao','preco_abertura','preco_maximo','preco_minimo',
          'preco_fechamento','qtd_negocios','volume_negocios']

  df = pd.read_fwf(_file, colspecs = colspecs, names = names, skiprows = 1 )
 
  return df

In [ ]:
# Filtrando ações do lote padrão(2)
def filter_stocks(df):
  df = df [df['codbdi'] == 2]
  df = df.drop(['codbdi'],1)
  return df

In [ ]:
# Ajuste campo de data
def parse_date(df):
  df['data_pregao'] = pd.to_datetime(df['data_pregao'], format = '%Y%m%d')
  return df

In [ ]:
# Ajuste dos campos numéricos 
def parse_values(df):
  df['preco_abertura'] = (df['preco_abertura']/100).astype(float)
  df['preco_maximo'] = (df['preco_maximo']/100).astype(float)
  df['preco_minimo'] = (df['preco_minimo']/100).astype(float)
  df['preco_fechamento'] = (df['preco_fechamento']/100).astype(float)
  
  return df

In [ ]:
# Juntando os arquivos

def concat_files(path, name_file, year_date, type_file, final_file):

  for i, y in enumerate(year_date):
    df = read_files(path, name_file, y, type_file)
    df = filter_stocks(df)
    df = parse_date(df)
    df = parse_values(df)

    if i==0:
      df_final = df
    else:
      df_final = pd.concat([df_final, df])

  df_final.to_csv(f'{path}//{final_file}', index= False)

###### Após carregar as bases de dados dos anos selecionados no site da bovespa, digite separado por virgulas e entre aspas os valores dos anos dos arquivos na várivavel year_date

In [ ]:
# Executando script
year_date = ['2018','2019','2020']

path ='/content/'

name_file = 'COTAHIST_A'

type_file = 'TXT'

final_file = 'all_bovespa.csv'

concat_files(path, name_file, year_date, type_file, final_file)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


###### Para fazer o download do arquivo após o processo de ETL só clicar novamente no icone 'files' clicar em cima do arquivo 'all_bovespa.csv' com o botão direito e selecionar o download